## Module 2B: Retail price optimization - Automated Implicit Model

[**Link to Google Colab of this Notebook**](https://colab.research.google.com/github/acedesci/scanalytics/blob/master/EN/S08_09_Retail_Analytics/S9_Module2B_Retail_Price_Optimization_Script.ipynb)

This notebook is the script version of the Module2A (explicit model). Unlike the explicit model in which we need to explicitly add each complete equation one by one, we can automate the model generation process by using a script version of it. For this one, we do not expect that you understand in detail how to generate the script but simply understand what each block does. Creating the script would require some experience. The main purpose of this is to provide an example of real-life mathematical programming workflow which automate the prescriptive analytics process.

More particularly, we want to create an **implicit (or compact) model** of the following prescriptive pricing model of Rue La La.

<img src="https://github.com/acedesci/scanalytics/blob/master/EN/S08_09_Retail_Analytics/implicit_model.png?raw=true" width="600">

The following blocks install Pyomo and solver. We also provide an option to use a more powerful solver *CBC* in addition to GLPK we used earlier. You can outcomment it if you want to switch to CBC.

```python
# Install Pyomo and GLPK in your Python environment first
pip install -q pyomo
conda install conda-forge::glpk
```

### Block 1: Data input

We prepared the data inputs in two files, i.e.,


1.   **'predictedSales_Prob1.csv'**. This is a small scale problem. It is identical to the problem you see in the Module 2A (explicit model).
2.   **'predictedSales_Prob2.csv'**. This is a large-scale problem. This one contains a much higher number of variables and constraints to reflect real-life setting.

Please mainly focus on the file *'predictedSales_Prob1.csv'* since you will get to see the same model as Module_1A. You can also try *'predictedSales_Prob2.csv'* if you are interested to see the large-scale model.

In order to read the input, we take the file from the URL. This is the same file that you would obtain if you run the module 1B. If you want, you can change this block so that you can upload it from your PC or load it from Google Drive (see also Module 1B how these two options can be done).



In [1]:
import pandas

# Prob1 is the same problem as Module 2A
url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/EN/S08_09_Retail_Analytics/predictedSales_Prob1.csv'

# Prob2 is the large-scale problem
# url = 'https://raw.githubusercontent.com/acedesci/scanalytics/master/EN/S08_09_Retail_Analytics/predictedSales_Prob2.csv'

predDemand = pandas.read_csv(url)

# Dataset is now stored in a Pandas Dataframe predDemand
predDemand

,NA,avgPriceChoice,UPC,PRICE,PRICE_p2,FEATURE,DISPLAY,TPR_ONLY,RELPRICE,predictSales
0,0,3.0,1600027528,2.5,6.25,0,0,0,0.833333,95.0
1,1,3.0,1600027528,3.0,9.00,0,0,0,1.000000,67.0
2,2,3.0,1600027528,3.5,12.25,0,0,0,1.166667,46.0
3,3,3.0,1600027564,2.5,6.25,0,0,0,0.833333,24.0
4,4,3.0,1600027564,3.0,9.00,0,0,0,1.000000,23.0
5,5,3.0,1600027564,3.5,12.25,0,0,0,1.166667,20.0
6,6,3.0,3000006340,2.5,6.25,0,0,0,0.833333,6.0
7,7,3.0,3000006340,3.0,9.00,0,0,0,1.000000,4.0
8,8,3.0,3000006340,3.5,12.25,0,0,0,1.166667,3.0
9,9,3.0,3800031829,2.5,6.25,0,0,0,0.833333,33.0


With the new dataset, we first need to check how many average price values are there because we need to run the optimization model for each value of the average price.

In [2]:
avgPriceList = predDemand['avgPriceChoice'].unique()
inputColumns = ['avgPriceChoice', 'UPC', 'PRICE','predictSales']
print("Possible average price choices (k/N.Product):"+str(avgPriceList))


Possible average price choices (k/N.Product):[3.]


### Block 2: Prepare input parameters for the model

We can choose which value of $k$ we want to use in the optimization model from the *avgPriceChoice* we have in the dataset. In *'predictedSales_Prob1.csv'*, there is only one average price choice at $3.0 whereas in *'predictedSales_Prob2.csv'* there are 4 different price choices you can choose form.

If you want to try different average price choices, we would need to repeat this procedure for each average price value and record the corresponding optimal solution to decide how each product should be priced and at which average price level to generate the optimal revenue.

Note that in this demo, we use $p_{ij}$ instead of $p_{j}$ since it is easier to prepare the script but the model remains identical to the Module 2A because $p_{ij} = p_{j},i=1,...,n$.

In [3]:
# Nere we choose which value of k (avgPriceValue x N. of products) we would like to use in the model
# Note that k must be among the choices where the prediction has been prepared
avgPriceValue =  avgPriceList[0]

# Now we select only the row which corresponds to the previously chosen value of avgPriceValue (again k = avgPriceValue x N. of products)
predDemand_k = predDemand.loc[predDemand['avgPriceChoice'] == avgPriceValue][inputColumns]
print(predDemand_k)
productList = predDemand_k['UPC'].unique()
priceList = predDemand_k['PRICE'].unique()

# Here we prepare the dictionary to be used in the optimization model
p = {}
D = {}

for upc in productList:
  for price in priceList:
    p[(upc,price)] = price
    D[(upc,price)] = predDemand_k.loc[(predDemand['UPC'] == upc) & (predDemand_k['PRICE'] == price)]['predictSales'].values[0]

print(p)
print(D)

    avgPriceChoice         UPC  PRICE  predictSales
0              3.0  1600027528    2.5          95.0
1              3.0  1600027528    3.0          67.0
2              3.0  1600027528    3.5          46.0
3              3.0  1600027564    2.5          24.0
4              3.0  1600027564    3.0          23.0
5              3.0  1600027564    3.5          20.0
6              3.0  3000006340    2.5           6.0
7              3.0  3000006340    3.0           4.0
8              3.0  3000006340    3.5           3.0
9              3.0  3800031829    2.5          33.0
10             3.0  3800031829    3.0          24.0
11             3.0  3800031829    3.5          20.0
{(1600027528, 2.5): 2.5, (1600027528, 3.0): 3.0, (1600027528, 3.5): 3.5, (1600027564, 2.5): 2.5, (1600027564, 3.0): 3.0, (1600027564, 3.5): 3.5, (3000006340, 2.5): 2.5, (3000006340, 3.0): 3.0, (3000006340, 3.5): 3.5, (3800031829, 2.5): 2.5, (3800031829, 3.0): 3.0, (3800031829, 3.5): 3.5}
{(1600027528, 2.5): 95.0, (16000275

### Block 3: Create an optimization model

#### Block 3.1: Variable declarations

Unlike the first part of today's session, we index the decision variables and demand parameters by the product and the price themselves rather than their index. Indeed, we previously denoted $x_{ij}=1$ if the price option $j$ is chosen for product $i$, and 0 otherwise. Now, our variable is denoted by $x_{1600027528,\ 3.0}=1$, which means that product UPC '1600027528' will be sold at 3.0 dollars. The same notational remark applies to predicted demand ($D_{ijk}$) for the sum of prices $k$ and price ($p_{ij}$) parameters. We can declare the constraint sets first (model.PriceChoiceUPC, model.sumPrice) and then **add** the constraint functions later.

In [4]:
from pyomo.environ import *

iIndexList = list(range(len(productList)))
jIndexList = list(range(len(priceList)))

model = ConcreteModel()
# Variables
model.x = Var(productList, priceList, within = Binary)

# Constraints
model.PriceChoiceUPC = ConstraintList()
model.sumPrice = ConstraintList()

# Print to review the model (equations are still not included)
model.pprint()

5 Set Declarations
    PriceChoiceUPC_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}
    sumPrice_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    0 :      {}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   12 : {(1600027528, 2.5), (1600027528, 3.0), (1600027528, 3.5), (1600027564, 2.5), (1600027564, 3.0), (1600027564, 3.5), (3000006340, 2.5), (3000006340, 3.0), (3000006340, 3.5), (3800031829, 2.5), (3800031829, 3.0), (3800031829, 3.5)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1600027528, 1600027564, 3000006340, 3800031829}
    x_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Member

#### Block 3.2: Adding an objective function

Instead of iteratively entering the value for each price and predicted sales, we can simply create a loop **for** each product and a loop **for** each price. The code now looks very much like the general equation $\sum_{i} \sum_{j} p_{ij} \cdot D_{ijk} \cdot x_{ij}$ we saw in the first part of today's session with some minor changes for notational simplification.

In [5]:
# Objective function

obj_expr = sum(p[(i,j)]*D[(i,j)]*model.x[i,j] for i in productList for j in priceList)
print(obj_expr)
model.OBJ = Objective(expr = obj_expr, sense = maximize)

237.5*x[1600027528,2.5] + 201.0*x[1600027528,3.0] + 161.0*x[1600027528,3.5] + 60.0*x[1600027564,2.5] + 69.0*x[1600027564,3.0] + 70.0*x[1600027564,3.5] + 15.0*x[3000006340,2.5] + 12.0*x[3000006340,3.0] + 10.5*x[3000006340,3.5] + 82.5*x[3800031829,2.5] + 72.0*x[3800031829,3.0] + 70.0*x[3800031829,3.5]


#### Block 3.3: Adding constraints

**Constraint 1: One price choice must be selected for each product**

Similarly, we can create a loop to **add** constraint functions to the constraint set **for** each product to ensure that only one price on the list is selected for that product. Unlike the first part of today's session, we need not iteratively type each constraint.

In [6]:
# Constraints #1
for i in productList:
  const1_expr = sum(model.x[i,j] for j in priceList) == 1
  print(const1_expr)
  model.PriceChoiceUPC.add(expr = const1_expr)


x[1600027528,2.5] + x[1600027528,3.0] + x[1600027528,3.5]  ==  1
x[1600027564,2.5] + x[1600027564,3.0] + x[1600027564,3.5]  ==  1
x[3000006340,2.5] + x[3000006340,3.0] + x[3000006340,3.5]  ==  1
x[3800031829,2.5] + x[3800031829,3.0] + x[3800031829,3.5]  ==  1


**Constraint 2: The sum of the prices of all products must equal $k$**

Similar **for** loops apply to the average price constraint. Please refer to the first part of today's session for detailed elaboration.

In [7]:
# Constraints #2
const2_expr = sum(p[i,j]*model.x[i,j] for i in productList for j in priceList) == avgPriceValue*len(productList)
print(const2_expr)
model.sumPrice.add(expr = const2_expr)



2.5*x[1600027528,2.5] + 3.0*x[1600027528,3.0] + 3.5*x[1600027528,3.5] + 2.5*x[1600027564,2.5] + 3.0*x[1600027564,3.0] + 3.5*x[1600027564,3.5] + 2.5*x[3000006340,2.5] + 3.0*x[3000006340,3.0] + 3.5*x[3000006340,3.5] + 2.5*x[3800031829,2.5] + 3.0*x[3800031829,3.0] + 3.5*x[3800031829,3.5]  ==  12.0


We can print the model to review prior to solving it.

In [8]:
model.pprint()

5 Set Declarations
    PriceChoiceUPC_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1, 2, 3, 4}
    sumPrice_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    1 :    {1,}
    x_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain              : Size : Members
        None :     2 : x_index_0*x_index_1 :   12 : {(1600027528, 2.5), (1600027528, 3.0), (1600027528, 3.5), (1600027564, 2.5), (1600027564, 3.0), (1600027564, 3.5), (3000006340, 2.5), (3000006340, 3.0), (3000006340, 3.5), (3800031829, 2.5), (3800031829, 3.0), (3800031829, 3.5)}
    x_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    4 : {1600027528, 1600027564, 3000006340, 3800031829}
    x_index_1 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : M

### Block 4: Solution and results

Finally, we call the solver and obtain the optimal solution. We can see that product '1600027528' is also sold at price $\$2.5$, products '1600027564' and '3000006340' both  at price $\$3.5$ and product '3800031829' at price $\$2.5$, but the optimal objective value is now $\$399.3$. The objective function value is slightly different from the Module_2A but the solution (values of $x$) is the same. This is due to the fact that we keep more digits in this example.

In [9]:
# Solve the model
opt = SolverFactory('glpk')
opt.solve(model)

model.display()

Model unknown

  Variables:
    x : Size=12, Index=x_index
        Key               : Lower : Value : Upper : Fixed : Stale : Domain
        (1600027528, 2.5) :     0 :   1.0 :     1 : False : False : Binary
        (1600027528, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (1600027528, 3.5) :     0 :   0.0 :     1 : False : False : Binary
        (1600027564, 2.5) :     0 :   0.0 :     1 : False : False : Binary
        (1600027564, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (1600027564, 3.5) :     0 :   1.0 :     1 : False : False : Binary
        (3000006340, 2.5) :     0 :   0.0 :     1 : False : False : Binary
        (3000006340, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (3000006340, 3.5) :     0 :   1.0 :     1 : False : False : Binary
        (3800031829, 2.5) :     0 :   1.0 :     1 : False : False : Binary
        (3800031829, 3.0) :     0 :   0.0 :     1 : False : False : Binary
        (3800031829, 3.5) :     0 :   0.0